# COMP41680 Assignment 2 - Part_1
Name: Shuhao Guan  
ID: 20211120

## Task 1. Data Collection

**1. Scrape the complete set of web pages from your personal website address:**
- http://mlg.ucd.ie/modules/python/assign2/20211120/ 

**2. From the web pages above, parse every review across all years 2016-2021. For each product review, extract the following information:** 

- The star rating of the review 
- The title text of the review 
- The main body text of the review 
- Review helpfulness information 

**3. Store the parsed review data in an appropriate format.**

### Import the library

In [1]:
import urllib.request
import bs4
import pandas as pd
import csv
import os

### Scrape all page names

In [2]:
link_ = "http://mlg.ucd.ie/modules/python/assign2/20211120/"
response_ = urllib.request.urlopen(link_)
html_ = response_.read().decode()
parser_ = bs4.BeautifulSoup(html_,"html.parser")
# start=int(parser2.find_all("div", class_="jumbotron")[0].p.get_text().split(" ")[5])
# end=int(parser2.find_all("div", class_="jumbotron")[0].p.get_text().split(" ")[7][:-1])
list=[]
for i in parser_.find_all("div", class_="list-group"):
    for j in i.find_all('a'):
        for k in range((int(str(j).split("[")[1][:-5])-1)//30+1):
#             print(str(j).split("[")[1][:-5])
            list.append(str(j).split("\"")[3][:-7]+str(k+1).zfill(2)+".html")

In [3]:
list[:5]

['reviews-2016-jan-01.html',
 'reviews-2016-jan-02.html',
 'reviews-2016-jan-03.html',
 'reviews-2016-jan-04.html',
 'reviews-2016-jan-05.html']

### Scrape all review information

In [4]:
def got(links):
    dfall=pd.DataFrame()
    for link in links:
#         print(link)
        response = urllib.request.urlopen("http://mlg.ucd.ie/modules/python/assign2/20211120/"+link)
        html = response.read().decode()
        parser = bs4.BeautifulSoup(html,"html.parser")
        # star
        star=[]
        stars = parser.find_all('img')
        for i in stars:
            star.append(int(i["alt"].split('-')[0]))
        # title
        title=[]
        titles = parser.find_all('h5')
        for i in titles:
            title.append(i.get_text().split('\xa0')[1])
        #helpful
        helpful1=[]
        helpful2=[]
        for i in range(1,len(parser.find_all("p", class_="metadata")),2):
        #     print(parser.find_all("p", class_="metadata")[i].get_text())
            helpful1.append(int(parser.find_all("p", class_="metadata")[i].get_text().split(" ")[0]))
            helpful2.append(int(parser.find_all("p", class_="metadata")[i].get_text().split(" ")[3]))
        # review
        review=[]
        reviews=parser.find_all("p", class_="review-body")
        for i in reviews:
            review.append(i.get_text())
        # parser.find_all("p", class_="review-body")[0].get_text()
        df=[star,title,review,helpful1,helpful2]
        df=pd.DataFrame(df)
        df=df.T
        # Python program to demonstrate
        # writing to CSV
        dfall=pd.concat([dfall, df])

        # field names
        fields = ['star','title','review','helpful1','helpful2']
        # name of csv file
        if not os.path.exists("./data"):
            os.makedirs("./data")
        filename = "./data/"+link.split(".")[0]+".csv"
        # writing to csv file
        with open(filename, 'w',encoding='utf-8') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile, lineterminator='\n')
            # writing the fields
            csvwriter.writerow(fields)
            # writing the data rows
            csvwriter.writerows(df.values)
    dfall.columns=['star','title','review','helpful1','helpful2']
    if not os.path.exists("./merged_data"):
        os.makedirs("./merged_data")
    dfall.to_csv("./merged_data/review.csv",index=False)

I have stored the separate data and the merged data separately.

### Implementation

In [5]:
got(list)